In [3]:
from functools import lru_cache, cached_property

class PromptCache:
    def __init__(self, cache_size: int = 200):
        self.cache_size = cache_size
        self._cache     = lru_cache(maxsize=cache_size)(self._raw_cache)()

    @cached_property
    def cache(self):
        return self._cache

    def get(self, key):
        return self.cache.get(key)

    def add(self, key, value):
        if key not in self.cache:
            self.cache[key] = value

    def clear(self):
        self.cache.clear()

    @staticmethod
    def _raw_cache():
        return {}

    
    
    
import os
from glob import glob
from typing import List
from jinja2 import Template, Environment, FileSystemLoader, meta


class TemplateLoader:
    """
    A class for loading and managing Jinja2 templates. It allows loading templates from files or strings,
    listing available templates, and getting template variables.
    """

    def __init__(self):
        """
        Initialize the TemplateLoader object and create an empty dictionary for loaded templates.
        """
        self.loaded_templates = {}

    def load_template(
        self, template: str, from_string: bool = False
    ):
        """
        Load a Jinja2 template either from a string or a file.

        Args:
            template (str): Template string or path to the template file.
            from_string (bool): Whether to load the template from a string. Defaults to False.

        Returns:
            dict: Loaded template data.
        """
        if template in self.loaded_templates:
            return self.loaded_templates[template]

        if from_string:
            template_instance = Template(template)
            template_data = {
                "template_name": "from_string",
                "template_dir": None,
                "environment": None,
                "template": template_instance,
            }
        else:
            template_data = self._load_template_from_path(template)

        self.loaded_templates[template] = template_data
        return self.loaded_templates[template]

    def _load_template_from_path(self, template: str) -> dict:
        """
        Load a Jinja2 template from the given path.

        Args:
            template (str): Path to the template file.

        Returns:
            dict: Loaded template data.
        """
        current_dir = os.path.dirname(os.path.realpath('.'))
        current_dir, _ = os.path.split(current_dir)
        templates_dir = os.path.join(current_dir, ".")
        all_folders = {
            f"{folder}.jinja": folder for folder in os.listdir(templates_dir)
        }

        if template in all_folders:
            
            template_name_, _ = template_name.split(".jinja")
            template_dir = os.path.join(templates_dir, template_name_)
            
            environment = Environment(loader=FileSystemLoader(template_dir))
            template_instance = environment.get_template(template_name)

        else:
            self._verify_template_path(template)
            custom_template_dir, custom_template_name = os.path.split(template)

            template_name = custom_template_name
            template_dir = custom_template_dir
            environment = Environment(loader=FileSystemLoader(template_dir))
            template_instance = environment.get_template(custom_template_name)

        return {
            "template_name": template_name,
            "template_dir": template_dir,
            "environment": environment,
            "template": template_instance,
        }

    def _get_metadata(self, template_name, template_path):
        
        template_name, _ = template_name.split(".jinja")
        metadata_files = glob(
            os.path.join(template_path, template_name, "metadata.json")
        )

        metadata = read_json(metadata_files[0])[0]
        metadata["file_path"] = os.path.join(template_path, template_name)
        return {"metadata": metadata}


    def _verify_template_path(self, templates_path: str):
        """
        Verify the existence of the template file.

        Args:
            templates_path (str): Path to the template file.

        Raises:
            ValueError: If the template file does not exist.
        """
        if not os.path.isfile(templates_path):
            raise ValueError(f"Templates path {templates_path} does not exist")

    def list_templates(self, environment) -> List[str]:
        """
        List all templates in the specified environment.

            Args:
                environment (Environment): The Jinja2 environment to search for templates.

        Returns:
            List[str]: List of available template names.
        """
        return environment.list_templates()

    def get_template_variables(self, environment, template_name) -> List[str]:
        """
        Get a list of undeclared variables for the specified template.

        Args:
            environment (Environment): The Jinja2 environment of the template.
            template_name (str): The name of the template.

        Returns:
            List[str]: List of undeclared variables in the template.
        """
        template_source = environment.loader.get_source(environment, template_name)
        parsed_content = environment.parse(template_source)
        return list(meta.find_undeclared_variables(parsed_content))


    

import json
import uuid
import datetime
from pathlib import Path
import hashlib
import os
import io


def read_json(json_file):
    """
    Reads JSON data from a file and returns a Python object.

    Args:
        json_file (str): The path to the JSON file to read.

    Returns:
        A Python object representing the JSON data.
    """
    with open(json_file) as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError as e:
            raise ValueError(
                f"Error decoding JSON data from file {json_file}: {str(e)}"
            )
    return data


def write_json(path, data, file_name):
    """
    Writes JSON data to a file.

    Args:
        path (str): The path to the directory where the file should be saved.
        data (Any): The data to write to the file. This can be any JSON-serializable object.
        file_name (str): The name of the file to write, without the '.json' extension.

    Raises:
        IOError: If there is a problem writing the file.
    """
    full_path = os.path.join(path, f"{file_name}.json")
    try:
        with open(full_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
    except IOError as e:
        raise IOError(f"Error writing JSON file '{full_path}': {e.strerror}")


def calculate_hash(text: str, encoding: str = "utf-8") -> str:
    """
    Calculate the hash of a text using the specified encoding.

    Args:
        text: The text to calculate the hash for.
        encoding: The encoding to use for the text. Defaults to "utf-8".

    Returns:
        The hash of the text.
    """
    if not isinstance(text, str):
        raise TypeError("Expected a string for 'text' parameter.")

    hash_obj = hashlib.md5()
    hash_obj.update(text.encode(encoding))
    return hash_obj.hexdigest()


def setup_folder(folder_path: str, folder_name: str = None) -> str:
    """
    Creates a folder in the specified folder_path.

    Parameters
    ----------
    folder_path : str
        The path to the directory where the folder will be created.
    folder_name : str, optional
        The name of the folder. If None, a name will be generated using the
        current date and time and a UUID.

    Returns
    -------
    The path to the created folder.
    """

    if folder_name is None:
        current_date = datetime.datetime.now().strftime("%Y_%m_%d:%H:%M:%S")
        conversation_id = uuid.uuid4()
        folder_name = f"{current_date}_{conversation_id}"
        folder_name = calculate_hash(folder_name)

    folder_path = Path(folder_path)
    folder_path.mkdir(parents=True, exist_ok=True)

    folder = folder_path / folder_name
    folder.mkdir(parents=True, exist_ok=True)

    return str(folder), folder_name





import os
import uuid
from glob import glob
import datetime
from pathlib import Path


# from promptify.utils.data_utils import *
# from promptify.prompter.template_loader import TemplateLoader


from typing import List, Dict, Any, Optional
from jinja2 import Environment, FileSystemLoader, meta, Template

class Prompter:
    
    """
    A class to generate and handle utils related to prompts
    """

    def __init__(
        self,
        allowed_missing_variables: Optional[List[str]] = None,
        default_variable_values: Optional[Dict[str, Any]] = None
    ) -> None:
        """
        Initialize Prompter with default or user-specified settings.

        Parameters
        ----------
        allowed_missing_variables : list of str, optional
            A list of variable names that are allowed to be missing from the template. Default is ['examples', 'description', 'output_format'].
        default_variable_values : dict of str: any, optional
            A dictionary mapping variable names to default values to be used in the template.
            If a variable is not found in the input dictionary or in the default values, it will be assumed to be required and an error will be raised. Default is an empty dictionary.
        """


        self.default_variable_values = default_variable_values or {}
        self.template_loader = TemplateLoader()

        self.allowed_missing_variables = [
            "examples",
            "description",
            "output_format",
        ]
        self.allowed_missing_variables.extend(allowed_missing_variables or [])

    def update_default_variable_values(self, new_defaults: Dict[str, Any]) -> None:
        self.default_variable_values.update(new_defaults)

    def generate(self, template, text_input, **kwargs) -> str:
        """
        Generates a prompt based on a template and input variables.

        Parameters
        ----------
        text_input : str
            The input text to use in the prompt.
        **kwargs : dict
            Additional variables to be used in the template.

        Returns
        -------
        str
            The generated prompt string.
        """

        loader = self.template_loader.load_template(
            template, kwargs.get("from_string", False)
        )

        kwargs["text_input"] = text_input
        
        print(loader)

        if loader["environment"]:
            variables = self.template_loader.get_template_variables(
                loader["environment"], loader["template_name"]
            )
            variables_dict = {
                temp_variable_: kwargs.get(temp_variable_, None)
                for temp_variable_ in variables
            }

            variables_missing = [
                variable
                for variable in variables
                if variable not in kwargs
                and variable not in self.allowed_missing_variables
                and variable not in self.default_variable_values
            ]
            
            
            # print(variables_missing)
            if variables_missing:
                raise ValueError(
                    f"Missing required variables in template {', '.join(variables_missing)}"
                )
        else:
            variables_dict = {"data": None}

        kwargs.update(self.default_variable_values)
        prompt = loader["template"].render(**kwargs).strip()
        
        if kwargs.get("verbose", False):
            print(prompt)
            
        return prompt, variables_dict


import json

def is_string_or_digit(obj):
    """
    Check if an object is a string or a digit (integer or float).

    Args:
        obj (any): The object to be checked.

    Returns:
        bool: True if the object is a string or a digit, False otherwise.

    Examples:
        >>> is_string_or_digit("hello")
        True
        >>> is_string_or_digit(123)
        True
        >>> is_string_or_digit(3.14)
        True
        >>> is_string_or_digit(True)
        False
    """
    return isinstance(obj, (str, int, float))


In [7]:
# prompt = """{%- if description is not none -%}
# {{ description }}
# {% endif -%}

# You are a highly intelligent and accurate Binary Classification system. You take Passage as input and classify that as either {{ label_0 }} or {{ label_1 }} Category. Your output format is only {{ output_format|default("[{'C':Category}]") }} form, no other form.

# {% if examples is defined and examples|length > 0 -%}
# Examples:
# {% for sentence, label in examples %}
# Input: {{ sentence }}
# Output: [{'C': '{{ label }}' }]
# {% endfor %}
# {% endif -%}

# Input: {{ text_input }}
# Output:"""

# prompt = """I ate Something I don't know what it is... Why do I keep Telling things about food and elon musk was acusing Google and PayPal"""

raw_prompt = "quick brown fox jump over"

sp = Prompter()
print(sp.generate('/Users/ankit.pal/Desktop/Promptfinal/Promptify/promptify/prompter/abcd.jinja', raw_prompt, labels = 'aa')[0])



{'template_name': 'abcd.jinja', 'template_dir': '/Users/ankit.pal/Desktop/Promptfinal/Promptify/promptify/prompter', 'environment': <jinja2.environment.Environment object at 0x10646ecd0>, 'template': <Template 'abcd.jinja'>}
['domain']


ValueError: Missing required variables in template domain

{'template_name': 'from_string', 'template_dir': None, 'environment': None, 'template': <Template memory:1481f0fa0>}
/Users/ankit.pal/Desktop/Promptfinal/Promptify/promptify/prompter/ner.jinja


In [61]:
!pwd

/Users/ankit.pal/Desktop/Promptfinal/Promptify/promptify/prompter


In [57]:
from jinja2 import Template, Environment, FileSystemLoader, meta


prompt = """{%- if description is not none -%}
{{ description }}
{% endif -%}

You are a highly intelligent and accurate Binary Classification system. You take Passage as input and classify that as either {{ label_0 }} or {{ label_1 }} Category. Your output format is only {{ output_format|default("[{'C':Category}]") }} form, no other form.

{% if examples is defined and examples|length > 0 -%}
Examples:
{% for sentence, label in examples %}
Input: {{ sentence }}
Output: [{'C': '{{ label }}' }]
{% endfor %}
{% endif -%}

Input: {{ text_input }}
Output:"""




# kwargs.update(self.default_variable_values)
promptas = Template(prompt).render().strip()
promptas

"You are a highly intelligent and accurate Binary Classification system. You take Passage as input and classify that as either  or  Category. Your output format is only [{'C':Category}] form, no other form.\n\nInput: \nOutput:"

In [58]:
promptas.environment

AttributeError: 'str' object has no attribute 'environment'

In [8]:
import json
import uuid
import datetime
from typing import Dict, Any


def get_conversation_schema(
    conversation_id: str, llm_name: str, **llm_metadata: Any
) -> Dict[str, Any]:
    """
    Constructs a conversation schema with the specified parameters.

    Args:
    - conversation_id: A string representing the unique identifier of the conversation.
    - llm_name: A string representing the name of the language model.
    - **llm_metadata: Optional additional metadata to associate with the language model.

    Returns:
    A dictionary representing the conversation schema.
    """
    # Remove any api_key from the kwargs to avoid potential security issues
    llm_metadata.pop("api_key", None)

    # Construct the conversation schema dictionary
    conversation_schema = {
        "conversation_id": conversation_id,
        "start_time": str(datetime.datetime.now().strftime("%Y_%m_%d:%H:%M:%S")),
        "llm": {"name": llm_name, "meta_data": llm_metadata},
        "participants": [
            {"name": "User", "is_bot": False},
            {"name": "Assistant", "is_bot": True},
        ],
        "messages": [],
    }

    return conversation_schema


def create_message(
    task: str,
    prompt: str,
    response: str,
    structured_response: Any,
    **template_metadata: Any
) -> Dict[str, Any]:
    """
    Creates a message dictionary with the specified parameters.

    Args:
    - task: A string representing the task the message is associated with.
    - prompt: A string representing the prompt that initiated the message.
    - response: A string representing the message response.
    - structured_response: A structured representation of the message response.
    - **template_metadata: Optional metadata to associate with the message.

    Returns:
    A dictionary representing the message.
    """
    # Get the current timestamp as a formatted string
    timestamp = str(datetime.datetime.now().strftime("%Y_%m_%d:%H:%M:%S"))
    prompt_id = str(uuid.uuid4())

    # Construct the message dictionary
    
    message = {
        "timestamp": timestamp,
        "prompt_id":prompt_id,
        "task": task,
        "template_meta_data": template_metadata,
        "prompt": prompt,
        "response": response,
        "structured_response": structured_response,
    }

    return message










class ConversationLogger:
    
    def __init__(self, conversation_path: str, llm_parameters: Dict):
        
        """Create a logger for a conversation.

        Args:
            conversation_path: The path to the folder where conversations will be stored.
            model_name: The name of the language model used in the conversation.
            model_dict: A dictionary containing metadata about the model.
        """
        
        
        self.conversation_id   = uuid.uuid4()
        self.storage_name      = f'/llm_responses/llm_session_{self.conversation_id}/'
        self.conversation_path = os.path.join(conversation_path, self.storage_name)
        
        folder_path = Path(self.conversation_path)
        folder_path.mkdir(parents=True, exist_ok=True)
        
        model_dict = {key: value for key, value in llm_parameters.items() if is_string_or_digit(value)}
        self.conversation = get_conversation_schema(self.conversation_id, llm_parameters['model'], **llm_parameters)
        # write_json(self.conversation_path, self.conversation, "history")

    def add_message(self, message: Dict[str, Any]):

        """Add a message to the conversation.
        Args:
            prompt: The prompt sent to the language model.
            response: The response generated by the language model.
            source: The source of the message ("user" or "model").
            metadata: Optional metadata about the message.
            **kwargs: Additional metadata to be added to the message.
        """
        
        message_id   = uuid.uuid4()
        self.conversation["messages"].append(message)
        write_json(self.conversation_file, self.conversation, message_id)

    def __repr__(self):
        return f"ConversationLogger(conversation_id={self.conversation_id}, conversation_path={self.conversation_path})"


In [9]:
yu = ConversationLogger('.', {'a':1})

OSError: [Errno 30] Read-only file system: '/llm_responses'